In [1]:
import csv
import numpy as np
import pandas as pd
# import geopandas as gpd
from IPython.display import Image
# from shapely.geometry import Point, Polygon
from math import factorial
import datetime
import time
import scipy
import scipy.signal
import os, os.path

from statsmodels.sandbox.regression.predstd import wls_prediction_std
from sklearn.linear_model import LinearRegression
from patsy import cr

# from pprint import pprint
import matplotlib.pyplot as plt
import seaborn as sb


import sys
start_time = time.time()

/Users/hn/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:

sys.path.append('/Users/hn/Documents/00_GitHub/Ag/remote_sensing/python/')

import remote_sensing_core as rc
import remote_sensing_core as rcp

In [3]:
given_county = "Grant"
SF_year = 2016
indeks = "EVI"
SG_params = 51
delt = 0.4

In [4]:
SG_win_size = int(SG_params / 10) # grab the first digit as window size
SG_order = SG_params % 10 # grab the second digit as poly. order

print("SG_params is {}.".format(SG_params))
print("SG_win_size is {} and SG_order is {}.".format(SG_win_size, SG_order))


SG_params is 51.
SG_win_size is 5 and SG_order is 1.


In [5]:
onset_cut = 0.5
offset_cut = 0.5

regularized = True


In [6]:
regular_data_dir = "/Users/hn/Documents/01_research_data/remote_sensing/test_aeolus_outputs/regularized/"
f_name = "01_Regular_filledGap_" + given_county + "_SF_" + str(SF_year) + "_" + indeks + ".csv"

In [7]:
data_dir = regular_data_dir
a_df = pd.read_csv(data_dir + f_name, low_memory=False)

In [8]:
a_df = a_df[a_df['county']== given_county] # Filter the given county; given_county
a_df = a_df[a_df['image_year']== SF_year]

a_df['human_system_start_time'] = pd.to_datetime(a_df.human_system_start_time.values).values
a_df['Date'] = pd.to_datetime(a_df.Date.values).values

In [9]:
# a_df['year'] = SF_year
#
# The following columns do not exist in the old data
#
if not('DataSrc' in a_df.columns):
    print ("_________________________________________________________")
    print ("Data source is being set to NA")
    a_df['DataSrc'] = "NA"

a_df = rc.initial_clean(df = a_df, column_to_be_cleaned = indeks)

### List of unique polygons
polygon_list = a_df['ID'].unique()

print ("_________________________________________________________")
print("polygon_list is of length {}.".format(len(polygon_list)))


_________________________________________________________
polygon_list is of length 15751.


In [10]:
SEOS_output_columns = ['ID', 'Acres', 'county', 'CropGrp', 'CropTyp', 'DataSrc', 'ExctAcr',
                       'IntlSrD', 'Irrigtn', 'LstSrvD', 'Notes', 'RtCrpTy', 'Shap_Ar',
                       'Shp_Lng', 'TRS', 'image_year', 'SF_year', 'doy', 'EVI',
                       'human_system_start_time', 'Date', 'system_start_time', 
                       'EVI_ratio', 'SOS', 'EOS', 'season_count']

#
# The reason I am multiplying len(a_df) by 4 is that we can have at least two
# seasons which means 2 SOS and 2 EOS. So, at least 4 rows are needed.
#
all_poly_and_SEOS = pd.DataFrame(data = None, 
                                 index = np.arange(4*len(a_df)), 
                                 columns = SEOS_output_columns)

pointer_SEOS_tab = 0
counter = 0


In [11]:
a_df = a_df[SEOS_output_columns[0:22]]

In [86]:
a_poly = polygon_list[4]
curr_field = a_df[a_df['ID']==a_poly].copy()
curr_field.reset_index(drop=True, inplace=True)
# curr_field.columns

In [87]:
if (not("human_system_start_time" in list(curr_field.columns))):
        curr_field = rc.add_human_start_time(curr_field)
        
curr_field.sort_values(by=['image_year', 'doy'], inplace=True)

In [88]:
year = SF_year
plant = curr_field['CropTyp'].unique()[0]

# Take care of names, replace "/" and "," and " " by "_"
plant = plant.replace("/", "_")
plant = plant.replace(",", "_")
plant = plant.replace(" ", "_")
plant = plant.replace("__", "_")

ID = a_poly

### 
###  There is a chance that a polygon is repeated twice?
###

X = curr_field['doy']
y = curr_field[indeks]

In [89]:
SG_pred = scipy.signal.savgol_filter(y, window_length= SG_win_size, polyorder=SG_order)

curr_field[indeks] = SG_pred

y_orchard = curr_field[curr_field['doy']>= 122]
y_orchard = y_orchard[y_orchard['doy'] <=305]
y_orchard_range = max(y_orchard[indeks]) - min(y_orchard[indeks])
y_orchard_range

0.3503187851404954

In [92]:
if y_orchard_range > 0.3:
    #######################################################################
    ###
    ###             find SOS and EOS, and add them to the table
    ###
    #######################################################################
    curr_field = rc.addToDF_SOS_EOS_White(pd_TS = curr_field, 
                                          VegIdx = indeks, 
                                          onset_thresh = onset_cut, 
                                          offset_thresh = offset_cut)

    ##
    ##  Kill false detected seasons 
    ##
    curr_field = rc.Null_SOS_EOS_by_DoYDiff(pd_TS = curr_field, min_season_length=40)

    #
    # extract the SOS and EOS rows 
    #
    SEOS = curr_field[(curr_field['SOS'] != 0) | curr_field['EOS'] != 0]
    # SEOS = SEOS.reset_index() # not needed really
    SOS_tb = curr_field[curr_field['SOS'] != 0]
    if len(SOS_tb) >= 2:
        SEOS["season_count"] = len(SEOS)
        all_poly_and_SEOS[pointer_SEOS_tab:(pointer_SEOS_tab+len(SEOS))] = SEOS.values
        pointer_SEOS_tab += len(SEOS)
    else:
        aaa = curr_field.iloc[0].values.reshape(1, len(curr_field.iloc[0]))
        aaa = np.append(aaa, [1])
        aaa = aaa.reshape(1, len(aaa))
        
        all_poly_and_SEOS.iloc[pointer_SEOS_tab:(pointer_SEOS_tab+1)] = aaa
        pointer_SEOS_tab += 1
else:
    # we did not add EVI_ratio, SOS, and EOS. So, we are missing these
    # columns in the data frame. So, use 666 as proxy
    aaa = np.append(curr_field.iloc[0], [666, 666, 666, 1])
    aaa = aaa.reshape(1, len(aaa))
    
    all_poly_and_SEOS.iloc[pointer_SEOS_tab:(pointer_SEOS_tab+1)] = aaa
    pointer_SEOS_tab += 1

counter += 1

In [94]:
all_poly_and_SEOS.head(10)

,ID,Acres,county,CropGrp,CropTyp,DataSrc,ExctAcr,IntlSrD,Irrigtn,LstSrvD,...,SF_year,doy,EVI,human_system_start_time,Date,system_start_time,EVI_ratio,SOS,EOS,season_count
0,83672_WSDA_SF_2016,4,Grant,Orchard,apple,producer,4.1814,2003/07/01 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.378917,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
1,87971_WSDA_SF_2016,145,Grant,Orchard,apple,wsda,144.989,2006/08/31 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.374042,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
2,82161_WSDA_SF_2016,9,Grant,Orchard,apple,wsda,9.13309,2006/08/31 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.391748,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
3,82820_WSDA_SF_2016,16,Grant,Orchard,cherry,producer,16.034,2003/07/01 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.364603,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
4,79317_WSDA_SF_2016,17,Grant,Orchard,cherry,producer,17.453,2003/07/01 00:00:00,micro-sprinkler,2014/06/05 00:00:00,...,2016,1,0.36501,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.268564,0,0,1
5,81195_WSDA_SF_2016,16,Grant,Orchard,cherry,producer,15.6824,2003/07/01 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.366471,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
6,86456_WSDA_SF_2016,14,Grant,Orchard,cherry,producer,13.7232,2003/07/01 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.388346,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
7,78904_WSDA_SF_2016,17,Grant,Orchard,cherry,wsda,16.9407,2003/07/01 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.308673,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,0.12915,0,0,1
8,85950_WSDA_SF_2016,30,Grant,Orchard,cherry,wsda,30.2013,2011/05/25 00:00:00,micro-sprinkler,2014/06/05 00:00:00,...,2016,1,0.300091,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1
9,82825_WSDA_SF_2016,1,Grant,Vineyard,"grape, wine",wsda,1.27103,2006/08/31 00:00:00,sprinkler,2014/06/05 00:00:00,...,2016,1,0.155635,2016-01-01 00:00:00,2016-01-01 00:00:00,1451689400000,666,666,666,1


In [ ]:
all_poly_and_SEOS.shape

In [ ]:
print (all_poly_and_SEOS.shape)
print (curr_field.shape)

In [ ]:
all_poly_and_SEOS.columns

In [ ]:
curr_field.columns

In [ ]:
np.array((curr_field.iloc[0].values.reshape(1, len(curr_field.iloc[0])), 666, 666, 666))

In [ ]:
len(np.array((curr_field.iloc[0].values, 666, 666, 666)))

In [ ]:
type(np.array(curr_field.iloc[0].values))

In [ ]:
aaa = np.append(curr_field.iloc[0], [666, 666, 666])
aaa = aaa.reshape(1, 25)

In [ ]:
all_poly_and_SEOS.iloc[pointer_SEOS_tab:(pointer_SEOS_tab+1)] = aaa

In [ ]:
aaa.shape

In [ ]:
all_poly_and_SEOS.iloc[pointer_SEOS_tab:(pointer_SEOS_tab+1)].shape

In [ ]:
aaa.shape

In [ ]:
all_poly_and_SEOS.head(2)

In [ ]:
type(np.append(curr_field.iloc[0], [666, 666, 666]))

In [ ]:
aaa.shape

In [ ]:
output_dir = "/data/hydro/users/Hossein/remote_sensing/04_noJump_Regularized_plt_tbl_SOSEOS/2Yrs_tables_regular/"

In [ ]:
out_name = output_dir + given_county + "_" + str(SF_year) + "_regular_SG_" + indeks + \
           "_Sav_win" + str(SG_win_size) + "_Order"  + str(SG_order) + ".csv"


In [ ]:
out_name